## MTNOY STOCK PRICE ANALYSIS

In [9]:
import requests
import pandas as pd
from config import settings
import sqlite3

#### API EXPLORATION TO EXTRACT THE DATA

In [10]:
ticker_symbol = "MTNOY"
output_size = "full"
data_type = "json"

url = ("https://learn-api.wqu.edu/1/data-services/alpha-vantage/query?"
       "function=TIME_SERIES_DAILY&"
       f"symbol={ticker}&"
       f"outputsize={output_size}&"
       f"datatype={data_type}&"
       f"apikey={settings.alpha_api_key}")

print(url)

https://learn-api.wqu.edu/1/data-services/alpha-vantage/query?function=TIME_SERIES_DAILY&symbol=MTNOY&outputsize=full&datatype=json&apikey=9f6f20dcb464ccb1684848b91cf1d9dca95f684ec5e5020327dd34b0295a2906e24213edb9ce6c64a6ac0d7738dbe8bd54ced07f6e3e77d6b37f9af43ab941e70fbe9696c11fc358aeda7e78aee795889aa900925fafe1627daff8ab02b487821c595e392aa03765a9d81091003c05be97f1f9472edbbe7bd63ea4e7443fa0ec


In [17]:
# get the response
response = requests.get(url=url)

# response json
response_data = response.json()

# response keys
keys = response_data.keys()

#print
print(type(response_data))
print(response.status_code)
print(keys)
# dir(response)

<class 'dict'>
200
dict_keys(['Meta Data', 'Time Series (Daily)'])


In [28]:
# # Extract `"Time Series (Daily)"` value from `response_data`

mtnoy_data = response_data["Time Series (Daily)"]

mtnoy_data["2023-12-27"]

{'1. open': '6.2700',
 '2. high': '6.4000',
 '3. low': '6.2600',
 '4. close': '6.4000',
 '5. volume': '22036'}

In [38]:
# Store in a DataFrame

df_mtnoy = pd.DataFrame().from_dict(mtnoy_data, orient="index", dtype=float)

df_mtnoy.index = pd.to_datetime(df_mtnoy.index)

# Name index "date"
df_mtnoy.index.name="date"

# Using list comprehension to rename the columns
df_mtnoy.columns = [c.split(". ")[1] for c in df_mtnoy.columns]


print(df_mtnoy.info())
df_mtnoy.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4144 entries, 2023-12-27 to 2007-07-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4144 non-null   float64
 1   high    4144 non-null   float64
 2   low     4144 non-null   float64
 3   close   4144 non-null   float64
 4   volume  4144 non-null   float64
dtypes: float64(5)
memory usage: 194.2 KB
None


,open,high,low,close,volume
date,,,,,
2023-12-27,6.27,6.40,6.26,6.400,22036.0
2023-12-26,6.10,6.27,5.97,6.192,19176.0
2023-12-22,6.21,6.28,6.15,6.150,6902.0
2023-12-21,6.17,6.21,6.17,6.210,255083.0
2023-12-20,6.01,6.15,6.01,6.105,30943.0


#### DATA EXPLORATION